In [1]:
import os
import sys
import re
import numpy as np
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import select_nodes, to_param_dict, to_param_list, insert_zero_params, rescale_manybody_params
from statmechlib.preprocessing import Trajectory, scale_configuration, pair_dist_cutoff, find_index
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, udif_print, u_core
from statmechlib.preprocessing import universal_eos, downselect
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box
from statmechlib.read_write import params_to_pickle

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [5]:
# stats_file = 'stats_marinica' # 'stats_samples'
# target_file = 'target_marinica' # 'target_samples'
params_file = 'marinica_params'
# stats_file = 'stats_manyknots' # 'stats_samples'
# target_file = 'target_manyknots' # 'target_samples'
# stats_file = 'stats_manymd' # 'stats_samples'
# target_file = 'target_manymd' # 'target_samples'
#stats_file = 'stats_manyall' # 'stats_samples'
#target_file = 'target_manyall' # 'target_samples'
stats_file = 'stats_all_samples' # 'stats_samples'
target_file = 'target_all_samples' # 'target_samples'
# stats_file = 'stats_everything' # 'stats_samples'
# target_file = 'target_everything' # 'target_samples'
stats_md_file = 'stats_onlymd' # 'stats_samples'
target_md_file = 'target_onlymd' # 'stats_samples'

In [6]:
with open(os.path.join(working, stats_file+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, target_file+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)
    
with open(os.path.join(working, stats_md_file+'.pickle'), 'rb') as fi:
    stats_md = pickle.load(fi)

with open(os.path.join(working, target_md_file+'.pickle'), 'rb') as fi:
    targets_md = pickle.load(fi)

with open(os.path.join(working, params_file + '.pickle'), 'rb') as fi:
    param_list = pickle.load(fi)  

In [7]:
# pair_knots = [2.22, 2.46, 2.5648975,  2.629795 ,  2.6946925,  2.8663175,  2.973045 ,
#         3.0797725,  3.5164725,  3.846445 ,  4.1764175,  4.700845 ,
#         4.8953   ,  5.089755 ,  5.3429525,  5.401695 ,  5.4604375]
# edens_knots = [ 2.22, 2.5,  2.6946925, 2.8663175, 3.1,  3.5,  4.5]

# pair_knots = [2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525, 5.401695, 5.4604375]

# pair_knots = [2.10, 2.22, 2.46, 2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525,5.401695,5.4604375, 5.54,5.58,5.62]             
#              5.18, 5.38, 5.58, 5.62]




#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.5]
#edens_knots = [2.5, 2.8663175000000001, 3.1, 3.5, 4.62]


# pair_knots = [2.5648975, 2.629795, 2.6946925, 2.8663175, 2.973045,
#               3.0797725, 3.5164725, 3.846445, 4.1764175, 4.700845,
#               4.8953, 5.089755, 5.3429525,5.401695,5.4604375]  
#edens_knots = [2.5, 3.1, 3.5, 4.9]


pair_knots = [2.10, 2.2200000000000002, 2.46,2.5648974999999998,2.6297950000000001,2.6946924999999999,2.8663175000000001,
   2.9730449999999999,3.0797724999999998,3.5164724999999999,3.8464450000000001,4.1764175000000003,4.7008450000000002,
   4.8952999999999998,5.0897550000000003,5.3429525,5.4016950000000001,5.4604375000000003,5.54,5.5800000000000001,
   5.6200000000000001]
edens_knots = [2.10, 2.22, 2.5, 2.8663175000000001, 3.1, 3.5, 4.62]

In [8]:
stats = downselect(stats_all, pair_knots, edens_knots)
print(stats['hyperparams'])
stats_select_md = downselect(stats_md, pair_knots, edens_knots)
print(stats_select_md['hyperparams'])

{u'pair': [2.1000000000000001, 2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003, 5.54, 5.5800000000000001, 5.6200000000000001], u'edens': [2.1000000000000001, 2.2200000000000002, 2.5, 2.8663175000000001, 3.1000000000000001, 3.5, 4.6200000000000001]}
{u'pair': [2.1000000000000001, 2.2200000000000002, 2.46, 2.5648974999999998, 2.6297950000000001, 2.6946924999999999, 2.8663175000000001, 2.9730449999999999, 3.0797724999999998, 3.5164724999999999, 3.8464450000000001, 4.1764175000000003, 4.7008450000000002, 4.8952999999999998, 5.0897550000000003, 5.3429525, 5.4016950000000001, 5.4604375000000003, 5.54, 5.5800000000000001, 5.6200000000000001], u'edens': [2.1000000000000001, 2.2200000000000002, 2.5, 2.8663175000000001, 3.1000000000

In [9]:
targets['md'] = targets_md['md']
stats['md'] = stats_select_md['md']

In [10]:
# for key in sorted(targets.keys()):
#     for box, xyz, ene in zip(targets[key]['box'], targets[key]['xyz'], stats[key]['energy']):
#         n = len(xyz)
#         vol = np.linalg.det(box)
#         corr = float(n*n)/vol
#         ene.append([corr])

In [11]:
for i, box in enumerate(targets['eos']['box']):
    print(i, box[0,0])

(0, 6.3724472324999999)
(1, 7.0096919557500001)
(2, 7.6469366790000013)
(3, 8.2841814022500007)
(4, 8.9214261255)
(5, 9.5586708487499994)
(6, 10.195915572000001)
(7, 10.833160295250002)
(8, 11.470405018499999)
(9, 12.107649741749999)
(10, 12.744894465)
(11, 13.382139188250001)
(12, 14.0193839115)
(13, 14.656628634749998)
(14, 15.293873357999999)
(15, 15.93111808125)
(16, 16.568362804499998)
(17, 17.205607527750001)
(18, 17.842852251)
(19, 18.480096974249999)
(20, 19.117341697499999)


In [12]:
9.5586708487499994/12.74489446*2.74


2.055000000806205

In [13]:
targets['eos_bcc']['box'] = targets['eos_bcc']['box'][0:5]
targets['eos_bcc']['xyz'] = targets['eos_bcc']['xyz'][0:5]
targets['eos_bcc']['temp'] = targets['eos_bcc']['temp'][0:5]
targets['eos_bcc']['beta'] = targets['eos_bcc']['beta'][0:5]
targets['eos_bcc']['energy'] = targets['eos_bcc']['energy'][0:5]
targets['eos_bcc']['forces'] = targets['eos_bcc']['forces'][0:5]
stats['eos_bcc']['energy'] = stats['eos_bcc']['energy'][0:5]
stats['eos_bcc']['forces'] = stats['eos_bcc']['forces'][0:5]

In [14]:
targets['eos']['box'] = targets['eos']['box'][5:9]
targets['eos']['xyz'] = targets['eos']['xyz'][5:9]
targets['eos']['temp'] = targets['eos']['temp'][5:9]
targets['eos']['beta'] = targets['eos']['beta'][5:9]
targets['eos']['energy'] = targets['eos']['energy'][5:9]
targets['eos']['forces'] = targets['eos']['forces'][5:9]
stats['eos']['energy'] = stats['eos']['energy'][5:9]
stats['eos']['forces'] = stats['eos']['forces'][5:9]

In [15]:
targets['relax']['box'][1] = targets['inf']['box'][0]
targets['relax']['xyz'][1] = targets['inf']['xyz'][0]
targets['relax']['temp'][1] = targets['inf']['temp'][0]
targets['relax']['beta'][1] = targets['inf']['beta'][0]
targets['relax']['energy'][1] = targets['inf']['energy'][0]
targets['relax']['forces'][1] = targets['inf']['forces'][0]
stats['relax']['energy'][1] = stats['inf']['energy'][0]
stats['relax']['forces'][1] = stats['inf']['forces'][0]

In [16]:
targets['relax']['energy']

[-1139.2,
 -2.9688679595213743e-239,
 -1136.4010143821092,
 -1136.7508187321096,
 -1190.2611772847656,
 -1127.1746971578905]

In [17]:
targets['eos_fcc']['weight'] = 0.0
targets['bain']['weight'] = 0.0
targets['eos']['weight'] = 0.1
targets['eos_bcc']['weight'] = 0.0
targets['relax']['weight'] = 0.2
targets['inf']['weight'] = 0.0
#targets['liq_5000K']['weight'] = 0.1

In [18]:
[(key, targets[key]['weight']) for key in targets.keys()]

[('bcc_npt_langevin_3700K', 1.0),
 ('eos', 0.1),
 ('vac_npt_langevin_2000K', 1.0),
 ('md', 1.0),
 ('inf', 0.0),
 ('relax', 0.2),
 ('screw_111_npt_langevin_2000K', 1.0),
 ('bcc_npt_langevin_300K', 1.0),
 ('eos_bcc', 0.0),
 ('liq_5000K', 1.0),
 ('fcc_npt_langevin_300K', 1.0),
 ('vacint', 1.0),
 ('eos_fcc', 0.0),
 ('i110_npt_langevin_2000K', 1.0),
 ('liq_3700K', 1.0),
 ('i111_npt_langevin_2000K', 1.0),
 ('bain', 0.0)]

In [19]:
targets['eos']['temp'] = [5000.0, 5000., 5000., 5000.]
targets['eos']['beta'] = [1/5000.0, 1/5000., 1/5000., 1/5000.]

In [25]:
start_params = [-4.1635311, -0.00364115525,
                 0.0, 4525.61445, 107.488634,-15.2604631,8.88626493,10.7120352,46.1310765,6.70641338,-7.14000617,-2.02174483,
   1.9897676,-1.35491449,-0.812592118,1.99278451,-0.484145757,-0.0478673818,-1.46201396,1.19810233,-0.00736954413,
   -0.00388205423,-0.00512566656,
  0.0, 3987.36133,85.8053818,31.7983469,-1.52861585,-1.77508182,0.207405945]

params_fix = start_params

In [26]:
multi_pars = [[0.0, 4525.61445, 107.488634,-15.2604631]]#, 0.0, 3987.36133,85.8053818]]
#[ -2.99496889e+03,  -2.53118134e+03,   3.51755721e+02, -6.72486102e+01,  -1.81831439e+03,   8.22866029e+09]
#multi_pars = [[ -1.08883118e-02,   2.40839380e+03,   3.27843914e+02, -6.27355201e+01,  -1.04441372e-03,  -4.86554222e+03]]

# multi_pars = [[ -4.20354715e+00,  -2.33102863e-03,   2.78559547e-02,
#         -3.69797280e+03,   9.28651669e+01,  -3.08290042e-02,
#         -3.42747549e+03]]

In [27]:
# {'hyperparams': {u'edens': [2.2200000000000002,
#    2.5,
#    2.8663175000000001,
#    3.1000000000000001,
#    3.5,
#    4.6200000000000001],
#   u'pair': [2.2200000000000002,
#    2.46,
#    2.5648974999999998,
#    2.6297950000000001,
#    2.6946924999999999,
#    2.8663175000000001,
#    2.9730449999999999,
#    3.0797724999999998,
#    3.5164724999999999,
#    3.8464450000000001,
#    4.1764175000000003,
#    4.7008450000000002,
#    4.8952999999999998,
#    5.0897550000000003,
#    5.3429525,
#    5.4016950000000001,
#    5.4604375000000003,
#    5.54,
#    5.5800000000000001,
#    5.6200000000000001]},
#  'params': {'edens': [3987.36133,
#    85.8053818,
#    31.7983469,
#    -1.52861585,
#    -1.77508182,
#    0.207405945],
#   'embed': [-4.1635311, -0.00364115525],
#   'lrcorr': [],
#   'pair': [4525.61445,
#    107.488634,
#    -15.2604631,
#    8.88626493,
#    10.7120352,
#    46.1310765,
#    6.70641338,
#    -7.14000617,
#    -2.02174483,
#    1.9897676,
#    -1.35491449,
#    -0.812592118,
#    1.99278451,
#    -0.484145757,
#    -0.0478673818,
#    -1.46201396,
#    1.19810233,
#    -0.00736954413,
#    -0.00388205423,
#    -0.00512566656]}}

In [28]:
def optimize_EAM_mp(pars, targets, stats, utot_func, params_fix=[]):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1, params_fix), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)

    return tuple([output[1], output[0]])

In [29]:
params_output = []

pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1, params_fix=params_fix)

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, params_fix=params_fix)

    # initial ordering
    print('Initial params:', multi_pars[-1])
    
    #distances = pool.map(get_sd, multi_pars)
    
    distances = [get_sd(multi_pars[-1], params_fix=params_fix)]
    
    optimal_params = zip(distances, multi_pars[-1:])
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(50):
        #optimal_params = pool.map(get_opt, m_pars)
        optimal_params = [get_opt(m_pars)]
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print("Iteration {}, best params: {}".format(i, best_params))

    print('Final Best params:', best_params)
    
    params_output.append(best_params)

pool.close()
pool.join()

('Initial params:', [0.0, 4525.61445, 107.488634, -15.2604631])
loss 0.0830624764469 0.0830624764469 0.0
('Best params:', [(0.083062476446946559, [0.0, 4525.61445, 107.488634, -15.2604631])])
('ieam:', 0)
loss 0.0830624764469 0.0830624764469 0.0
loss 0.0830624764381 0.0830624764381 0.0
loss 0.0820808277676 0.0820808277676 0.0
loss 0.082699035053 0.082699035053 0.0
loss 0.0831635848567 0.0831635848567 0.0
loss 0.0822878069456 0.0822878069456 0.0
loss 0.0819980563993 0.0819980563993 0.0
loss 0.081451945556 0.081451945556 0.0
loss 0.0811753714078 0.0811753714078 0.0
loss 0.080188683181 0.080188683181 0.0
loss 0.0802806824114 0.0802806824114 0.0
loss 0.0796810480482 0.0796810480482 0.0
loss 0.0782990371754 0.0782990371754 0.0
loss 0.0779411845327 0.0779411845327 0.0
loss 0.0756799239629 0.0756799239629 0.0
loss 0.0756331029816 0.0756331029816 0.0
loss 0.0723665659559 0.0723665659559 0.0
loss 0.0728518631171 0.0728518631171 0.0
loss 0.0687825639906 0.0687825639906 0.0
loss 0.061921232469 0.

loss 0.000103058709344 0.000103058709344 0.0
loss 0.00010324622712 0.00010324622712 0.0
loss 0.000103086360141 0.000103086360141 0.0
loss 0.000101840270546 0.000101840270546 0.0
loss 0.000101112245795 0.000101112245795 0.0
loss 0.000102104288082 0.000102104288082 0.0
loss 0.000101008036914 0.000101008036914 0.0
loss 0.000100862627979 0.000100862627979 0.0
loss 0.000100830591663 0.000100830591663 0.0
loss 0.00010210705891 0.00010210705891 0.0
loss 0.000100323509904 0.000100323509904 0.0
loss 0.000102101196221 0.000102101196221 0.0
loss 0.000100821793063 0.000100821793063 0.0
loss 0.000100558796435 0.000100558796435 0.0
loss 0.000100365290045 0.000100365290045 0.0
loss 0.000101942578326 0.000101942578326 0.0
loss 0.000100071429294 0.000100071429294 0.0
loss 0.000101234460419 0.000101234460419 0.0
loss 0.000100124113309 0.000100124113309 0.0
loss 0.00010055871242 0.00010055871242 0.0
loss 0.000100082487765 0.000100082487765 0.0
loss 0.000100512966551 0.000100512966551 0.0
loss 0.000100016

loss 9.9908425295e-05 9.9908425295e-05 0.0
loss 9.99084252933e-05 9.99084252933e-05 0.0
loss 9.99084252952e-05 9.99084252952e-05 0.0
loss 9.99084252943e-05 9.99084252943e-05 0.0
loss 9.99084252951e-05 9.99084252951e-05 0.0
loss 9.99084252938e-05 9.99084252938e-05 0.0
loss 9.99084252949e-05 9.99084252949e-05 0.0
loss 9.99084252952e-05 9.99084252952e-05 0.0
loss 9.99084252925e-05 9.99084252925e-05 0.0
loss 9.9908425294e-05 9.9908425294e-05 0.0
loss 9.99084252926e-05 9.99084252926e-05 0.0
loss 9.99084252941e-05 9.99084252941e-05 0.0
Iteration 0, best params: [(9.9908425292511758e-05, array([  9.67703260e-03,   1.66603383e+04,   4.12551128e+02,
        -9.24069324e+01]))]
loss 9.99084252925e-05 9.99084252925e-05 0.0
loss 9.99084253044e-05 9.99084253044e-05 0.0
loss 0.000467374302552 0.000467374302552 0.0
loss 0.000164234123384 0.000164234123384 0.0
loss 0.000114091091456 0.000114091091456 0.0
loss 0.000434728900042 0.000434728900042 0.0
loss 0.000171489325664 0.000171489325664 0.0
loss 0.0

loss 9.99084114349e-05 9.99084114349e-05 0.0
loss 9.99084117326e-05 9.99084117326e-05 0.0
loss 9.99084080783e-05 9.99084080783e-05 0.0
loss 9.99084055417e-05 9.99084055417e-05 0.0
loss 9.99084038011e-05 9.99084038011e-05 0.0
loss 9.9908399252e-05 9.9908399252e-05 0.0
loss 9.99084010301e-05 9.99084010301e-05 0.0
loss 9.99084006242e-05 9.99084006242e-05 0.0
loss 9.99083965567e-05 9.99083965567e-05 0.0
loss 9.99083985227e-05 9.99083985227e-05 0.0
loss 9.99083942623e-05 9.99083942623e-05 0.0
loss 9.99083934112e-05 9.99083934112e-05 0.0
loss 9.99083941936e-05 9.99083941936e-05 0.0
loss 9.99083886169e-05 9.99083886169e-05 0.0
loss 9.99083853719e-05 9.99083853719e-05 0.0
loss 9.99083943452e-05 9.99083943452e-05 0.0
loss 9.99083968525e-05 9.99083968525e-05 0.0
loss 9.99083917793e-05 9.99083917793e-05 0.0
loss 9.99083966554e-05 9.99083966554e-05 0.0
loss 9.99083910296e-05 9.99083910296e-05 0.0
loss 9.99083847439e-05 9.99083847439e-05 0.0
loss 9.99083805447e-05 9.99083805447e-05 0.0
loss 9.99083

loss 9.87066784406e-05 9.87066784406e-05 0.0
loss 9.87044265648e-05 9.87044265648e-05 0.0
loss 9.87405140028e-05 9.87405140028e-05 0.0
loss 9.86615631646e-05 9.86615631646e-05 0.0
loss 9.86577943014e-05 9.86577943014e-05 0.0
loss 9.86215409513e-05 9.86215409513e-05 0.0
loss 9.85908139183e-05 9.85908139183e-05 0.0
loss 9.87023954499e-05 9.87023954499e-05 0.0
loss 9.85830235493e-05 9.85830235493e-05 0.0
loss 9.85527095164e-05 9.85527095164e-05 0.0
loss 9.86791218406e-05 9.86791218406e-05 0.0
loss 9.84820704577e-05 9.84820704577e-05 0.0
loss 9.83975323951e-05 9.83975323951e-05 0.0
loss 9.85851675748e-05 9.85851675748e-05 0.0
loss 9.8387944712e-05 9.8387944712e-05 0.0
loss 9.83396221648e-05 9.83396221648e-05 0.0
loss 9.83113723446e-05 9.83113723446e-05 0.0
loss 9.82788994497e-05 9.82788994497e-05 0.0
loss 9.82038938385e-05 9.82038938385e-05 0.0
loss 9.82246501445e-05 9.82246501445e-05 0.0
loss 9.80025297272e-05 9.80025297272e-05 0.0
loss 9.78642404458e-05 9.78642404458e-05 0.0
loss 9.81918

...

loss 9.64195917381e-05 9.64195917381e-05 0.0
loss 9.64227440552e-05 9.64227440552e-05 0.0
loss 9.64197201044e-05 9.64197201044e-05 0.0
loss 9.64199591515e-05 9.64199591515e-05 0.0
loss 9.64210613842e-05 9.64210613842e-05 0.0
loss 9.64192940208e-05 9.64192940208e-05 0.0
loss 9.64204785234e-05 9.64204785234e-05 0.0
loss 9.64191986918e-05 9.64191986918e-05 0.0
loss 9.64194962489e-05 9.64194962489e-05 0.0
loss 9.64197368908e-05 9.64197368908e-05 0.0
loss 9.64190991784e-05 9.64190991784e-05 0.0
loss 9.64199198779e-05 9.64199198779e-05 0.0
loss 9.64190500486e-05 9.64190500486e-05 0.0
loss 9.6419713453e-05 9.6419713453e-05 0.0
loss 9.64190053436e-05 9.64190053436e-05 0.0
loss 9.6419333402e-05 9.6419333402e-05 0.0
loss 9.64190063812e-05 9.64190063812e-05 0.0
loss 9.641910594e-05 9.641910594e-05 0.0
loss 9.64189908069e-05 9.64189908069e-05 0.0
loss 9.64191045681e-05 9.64191045681e-05 0.0
loss 9.64189739004e-05 9.64189739004e-05 0.0
loss 9.64190058696e-05 9.64190058696e-05 0.0
loss 9.64189668546

Process PoolWorker-12:
Process PoolWorker-10:
Process PoolWorker-11:
Process PoolWorker-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run


loss 9.64189517828e-05 9.64189517828e-05 0.0


  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
    self._target(*self._args, **self._kwargs)
  File "//anaconda/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "//anaconda/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    self._target(*self._args, **self._kwargs)
  File "//anaconda/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "//anaconda/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
    task = get()
  File "//anaconda/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
  File "//anaconda/lib/python2.7/multiprocessing/queues.py", line 376, in get
  File "//anaconda/lib/python2.7/multiprocessing/queues.py", line 378, in get
  File "//anaconda/lib/python2.7/multiprocessing/queues.

KeyboardInterrupt: 

In [30]:
# params_output = [ -4.72546533e+00,  -6.97068121e-03,  -3.51290250e+02,
#          1.63175272e+02,   9.43598046e+00,  -3.25684355e+00,
#          2.30446047e+01,   4.06678582e+01,   7.56509137e+00,
#         -7.51663487e+00,  -1.59146529e+00,   1.14098894e+00,
#         -8.25274736e-01,  -8.09954101e-01,   2.09984210e+00,
#         -6.52067950e-01,  -8.56530962e-02,  -1.49919134e+00,
#          1.27499075e+00,   1.07226915e+02,   2.36918896e+01,
#         -1.29592031e+00,  -1.49967198e+00,   2.14664790e-01]
# params_output = [[ -4.65804603e+00,  -7.56457588e-03,  -1.68777520e+02,
#          1.56052349e+02,   1.42649908e+01,  -4.55941009e+00,
#          1.97613340e+01,   4.84204936e+01,   6.49466770e+00,
#         -6.62233324e+00,  -2.16547494e+00,   1.30932417e+00,
#         -8.33281480e-01,  -8.32333597e-01,   2.09417158e+00,
#         -6.19566076e-01,  -9.53112547e-02,  -1.49756931e+00,
#          1.27167940e+00,   1.06853821e+02,   2.74234479e+01,
#         -1.26300179e+00,  -1.66378666e+00,   2.24848394e-01]]

# params_output = [[ -4.20657287e+00,  -4.79083415e-03,  -4.28662319e+03,
#          1.39452507e+02,   2.91473879e+01,  -1.08844243e+01,
#          1.59102238e+01,   4.86110415e+01,   7.30591870e+00,
#         -8.24225202e+00,  -1.76541035e+00,   1.69386336e+00,
#         -1.23281961e+00,  -7.91007042e-01,   1.98078659e+00,
#         -4.92223582e-01,  -4.64721217e-02,  -1.47436944e+00,
#          1.19244103e+00,  -3.03716289e+03,   1.25775067e+02,
#          3.27634816e+01,  -1.80274854e+00,  -1.77552231e+00,
#          2.10521327e-01]]

# params_output = [[ -4.18519141e+00,  -3.73537266e-03,   3.96171752e+03,
#          1.18316128e+02,  -8.46735170e+00,   1.49884675e+01,
#          5.43882551e+00,   4.19584851e+01,   1.05383775e+01,
#         -8.44501744e+00,  -1.76769900e+00,   2.25806651e+00,
#         -1.68693854e+00,  -4.53202950e-01,   1.47888419e+00,
#         -4.72027075e-01,  -7.36382597e-02,  -5.90831325e-01,
#          5.77070122e-01,   3.49058280e+03,   9.94638189e+01,
#          2.96556203e+01,  -7.14356623e-01,  -1.77478736e+00,
#          2.05018897e-01]] # s2 = 1.5313539642322428e-06

# params_output = [[ -4.19976555e+00,  -3.91095024e-03,   4.11762114e+03,
#          1.06506552e+02,   5.29253551e-01,   1.24324874e+00,
#          1.19653127e+01,   4.68002406e+01,   6.67699247e+00,
#         -7.04927872e+00,  -2.05647856e+00,   1.95042373e+00,
#         -1.33520940e+00,  -8.11118459e-01,   1.99354705e+00,
#         -4.84402488e-01,  -4.71892464e-02,  -1.46523753e+00,
#          1.19717936e+00,  -6.80779121e-03,  -3.49337274e-03,
#         -3.76805537e-03,   3.51929579e+03,   9.27623682e+01,
#          3.15808969e+01,  -1.44019460e+00,  -1.78665037e+00,
#          2.05532639e-01]]

# params_output = [[ -4.16353110e+00,  -3.64115525e-03,   4.52561445e+03,
#          1.07488634e+02,  -1.52604631e+01,   8.88626493e+00,
#          1.07120352e+01,   4.61310765e+01,   6.70641338e+00,
#         -7.14000617e+00,  -2.02174483e+00,   1.98976760e+00,
#         -1.35491449e+00,  -8.12592118e-01,   1.99278451e+00,
#         -4.84145757e-01,  -4.78673818e-02,  -1.46201396e+00,
#          1.19810233e+00,  -7.36954413e-03,  -3.88205423e-03,
#         -5.12566656e-03,   3.98736133e+03,   8.58053818e+01,
#          3.17983469e+01,  -1.52861585e+00,  -1.77508182e+00,
#          2.07405945e-01]]

# params_output = [[ -4.08460829e+00,  -1.38588599e-03,  -1.32507730e+04,
#          1.93928425e+03,   2.05970852e+01,  -2.37669887e+02,
#          1.21943656e+02,  -3.24296128e+01,   6.26409270e+01,
#          8.36697799e+00,  -9.54708966e+00,  -2.06899378e+00,
#          1.75611005e+00,  -1.35952781e+00,  -7.35465370e-01,
#          1.98598708e+00,  -4.73450090e-01,  -4.99940237e-02,
#         -1.45135718e+00,   1.19557147e+00,   1.75773867e-04,
#         -1.84569288e-02,  -2.63221509e-02,   1.65754590e+03,
#         -1.03315521e+02,   3.86317998e+01,  -1.37580192e+00,
#         -2.15109594e+00,   2.00163255e-01]]

# params_output = [[ -5.94255002e+00,  -4.52753307e-03,  -8.98973159e+01,
#          1.58307190e+02,  -7.12304117e+01,   6.46096124e+00,
#          3.42388839e-01,   6.02193303e+00,   1.71026053e+00,
#          8.61616548e-01,  -1.24021132e+00,  -7.52778245e-01,
#          2.16478392e+00,  -6.80500465e-01,   8.35831474e-03,
#         -1.70858135e+00,   1.33828368e+00,   1.06188317e+01,
#          2.26603603e+00,  -1.52122221e-02,   2.00566458e-02]]

# params_output.append([ -6.67485922e+00,  -1.02376651e-02,   1.91540112e+02,
#         -1.02318453e+02,   3.55386850e+01,  -5.52132877e+00,
#          9.33351478e+00,   9.88401954e-01,   1.50327402e+00,
#         -5.01846513e-01,  -1.06459763e+00,  -5.05974054e-01,
#          2.00937835e+00,  -6.02252894e-01,   8.10245260e-04,
#         -1.64969075e+00,   1.29187809e+00,   4.66926359e+01,
#          2.30114335e+00,  -7.42024973e-01,   4.07443544e-02])

# params_output.append([ -6.02193727e+00,  -3.85371079e-03,  -6.97726900e+01,
#          1.34506659e+02,  -5.71483113e+01,   1.76207446e+00,
#          7.47456613e+00,  -6.72890122e-01,   2.76103862e+00,
#          7.76284266e-01,  -1.27164334e+00,  -7.67755950e-01,
#          2.18615528e+00,  -6.83040984e-01,   2.38050659e-03,
#         -1.70140052e+00,   1.33115736e+00,   1.35592664e+01,
#          1.60373079e+00,   1.47525215e-01,   1.65277513e-02])

# params_output.append([ -5.85430083e+00,  -5.22915244e-03,  -3.84070447e+01,
#          1.18120884e+02,  -5.29405790e+01,   1.27051644e+00,
#          8.33161715e+00,  -2.35387541e+00,   3.06151106e+00,
#          8.07273686e-01,  -1.28106541e+00,  -7.86145231e-01,
#          2.20500009e+00,  -6.83596584e-01,  -3.70671719e-03,
#         -1.70638158e+00,   1.33597525e+00,   2.23939837e+01,
#          1.33742607e+00,   2.58862865e-01,   1.55687124e-02])

# params_output = [[ -4.54515300e+00,  -1.40465463e-03,  -5.12693248e+01,
#          1.21701242e+02,  -5.60593282e+01,   3.74156510e+00,
#          6.20500982e+00,  -3.94899725e+00,   4.55745011e+00,
#          2.57158211e-01,  -1.12342686e+00,  -7.22333730e-01,
#          1.98804201e+00,  -5.43139769e-01,  -1.88003107e-03,
#         -1.63718004e+00,   1.25050682e+00,   2.44423191e+01,
#          1.44283371e+00,   8.22580153e-01,   1.59596058e-02]]


# params_output = [[-4.20354715e+00,  -2.33102863e-03,
#                  2.78559547e-02, -3.69797280e+03,   9.28651669e+01,
#                   -15.2604631,8.88626493,10.7120352,46.1310765,6.70641338,-7.14000617,-2.02174483,
#    1.9897676,-1.35491449,-0.812592118,1.99278451,-0.484145757,-0.0478673818,-1.46201396,1.19810233,-0.00736954413,
#    -0.00388205423,-0.00512566656,
# -3.08290042e-02,-3.42747549e+03,
#                   85.8053818,31.7983469,-1.52861585,-1.77508182,0.207405945
#                 ]]

# [ -4.20354715e+00,  -2.33102863e-03,   2.78559547e-02, -3.69797280e+03,   9.28651669e+01,  -3.08290042e-02,
#         -3.42747549e+03]

params_output = [[-4.1635311, -0.00364115525,
                 9.67703260e-03,   1.66603383e+04,   4.12551128e+02,
        -9.24069324e+01, 8.88626493,10.7120352,46.1310765,6.70641338,-7.14000617,-2.02174483,
   1.9897676,-1.35491449,-0.812592118,1.99278451,-0.484145757,-0.0478673818,-1.46201396,1.19810233,-0.00736954413,
   -0.00388205423,-0.00512566656,
  0.0, 3987.36133,85.8053818,31.7983469,-1.52861585,-1.77508182,0.207405945]]



print(params_output)

[[-4.1635311, -0.00364115525, 0.0096770326, 16660.3383, 412.551128, -92.4069324, 8.88626493, 10.7120352, 46.1310765, 6.70641338, -7.14000617, -2.02174483, 1.9897676, -1.35491449, -0.812592118, 1.99278451, -0.484145757, -0.0478673818, -1.46201396, 1.19810233, -0.00736954413, -0.00388205423, -0.00512566656, 0.0, 3987.36133, 85.8053818, 31.7983469, -1.52861585, -1.77508182, 0.207405945]]


In [31]:
# convert lists of optimal parameters to dictionaries
params_dicts = []
for params in params_output:
    params_dicts.append(to_param_dict(params, stats['hyperparams']))

In [32]:
params_dicts

[{'hyperparams': {u'edens': [2.1000000000000001,
    2.2200000000000002,
    2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.6200000000000001],
   u'pair': [2.1000000000000001,
    2.2200000000000002,
    2.46,
    2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003,
    5.54,
    5.5800000000000001,
    5.6200000000000001]},
  'params': {'edens': [0.0,
    3987.36133,
    85.8053818,
    31.7983469,
    -1.52861585,
    -1.77508182,
    0.207405945],
   'embed': [-4.1635311, -0.00364115525],
   'lrcorr': [],
   'pair': [0.0096770326,
    16660.3383,
    412.551128,
    -92.4069324,
    8.88626493,
    10.7120352,
    46.1310765,
    6.70641338,
    -7.14000617,
    -2.02174483

In [33]:
file_name = os.path.join(working, "params_out.pickle")

for params in params_dicts:
    params_to_pickle(params, file_name)

In [33]:
with open(os.path.join(working, 'params_out' + '.pickle'), 'rb') as fi:
    retreived_params = pickle.load(fi)  

In [34]:
retreived_params

[{'hyperparams': {u'edens': [2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.5],
   u'pair': [2.2200000000000002,
    2.46,
    2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [106.853821,
    27.4234479,
    -1.26300179,
    -1.66378666,
    0.224848394],
   'embed': [-4.65804603, -0.00756457588],
   'lrcorr': [],
   'pair': [-168.77752,
    156.052349,
    14.2649908,
    -4.55941009,
    19.761334,
    48.4204936,
    6.4946677,
    -6.62233324,
    -2.16547494,
    1.30932417,
    -0.83328148,
    -0.832333597,
    2.09417158,
    -0.619566076,
    -0.0953112547,
    -1.49756931,
    1.2716794]}},
 {'hyperparams': {u'edens': [2.2200000000000002,

In [35]:
retreived_params[-2:]

[{'hyperparams': {u'edens': [2.5, 3.1000000000000001, 3.5, 4.9000000000000004],
   u'pair': [2.5648974999999998,
    2.6297950000000001,
    2.6946924999999999,
    2.8663175000000001,
    2.9730449999999999,
    3.0797724999999998,
    3.5164724999999999,
    3.8464450000000001,
    4.1764175000000003,
    4.7008450000000002,
    4.8952999999999998,
    5.0897550000000003,
    5.3429525,
    5.4016950000000001,
    5.4604375000000003]},
  'params': {'edens': [14.2447861, 0.868030511, 0.50452072, 0.00877385016],
   'embed': [-5.9173299, -0.00439326285],
   'lrcorr': [],
   'pair': [-48.6300768,
    122.549311,
    -60.4621784,
    9.09485341,
    -0.79271282,
    -0.309711402,
    4.12680657,
    0.540819962,
    -1.23317275,
    -0.795900416,
    2.17660195,
    -0.639345527,
    -0.00230178698,
    -1.7138732,
    1.32277953]}},
 {'hyperparams': {u'edens': [2.1000000000000001,
    2.2200000000000002,
    2.5,
    2.8663175000000001,
    3.1000000000000001,
    3.5,
    4.620000000000